# Identifying Parts of Speech and Named Entities Practical

### Import the necessary packages

In [1]:
import nltk
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
import pandas as pd
import matplotlib.pyplot as plt

### Load Data

In [2]:
bbc_data = pd.read_csv("csv_files/bbc_news.csv")

In [3]:
bbc_data.head()

Unnamed: 0  index                                              title  \
0           0   6684                              Can I refuse to work?   
1           1   9267  'Liz Truss the Brief?' World reacts to UK poli...   
2           2   7387  Rationing energy is nothing new for off-grid c...   
3           3    767  The hunt for superyachts of sanctioned Russian...   
4           4   3712  Platinum Jubilee: 70 years of the Queen in 70 ...   

                         pubDate  \
0  Wed, 10 Aug 2022 15:46:18 GMT   
1  Mon, 17 Oct 2022 11:35:12 GMT   
2  Wed, 31 Aug 2022 05:20:18 GMT   
3  Tue, 22 Mar 2022 14:37:01 GMT   
4  Wed, 01 Jun 2022 23:17:33 GMT   

                                                guid  \
0       https://www.bbc.co.uk/news/business-62147992   
1          https://www.bbc.co.uk/news/world-63285480   
2  https://www.bbc.co.uk/news/uk-scotland-highlan...   
3                https://www.bbc.co.uk/news/60739336   
4             https://www.bbc.co.uk/news/uk-61660128   

                                                link  \
0  https://www.bbc.co.uk/news/business-62147992?a...   
1  https://www.bbc.co.uk/news/world-63285480?at_m...   
2  https://www.bbc.co.uk/news/uk-scotland-highlan...   
3  https://www.bbc.co.uk/news/60739336?at_medium=...   
4  https://www.bbc.co.uk/news/uk-61660128?at_medi...   

                                         description  
0  With much of the UK enduring another period of...  
1  The UK's political chaos has been watched arou...  
2  Scoraig in the north west Highlands has long h...  
3  Wealthy Russians sanctioned by the US, EU and ...  
4  A quick look back at the Queen's 70 years on t...

In [4]:
bbc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   index        1000 non-null   int64 
 2   title        1000 non-null   object
 3   pubDate      1000 non-null   object
 4   guid         1000 non-null   object
 5   link         1000 non-null   object
 6   description  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [5]:
titles = pd.DataFrame(bbc_data['title'])

In [6]:
titles.head()

title
0                              Can I refuse to work?
1  'Liz Truss the Brief?' World reacts to UK poli...
2  Rationing energy is nothing new for off-grid c...
3  The hunt for superyachts of sanctioned Russian...
4  Platinum Jubilee: 70 years of the Queen in 70 ...

### Clean Data

In [7]:
# lowercase
titles['lowercase'] = titles['title'].str.lower()
titles.head()

title  \
0                              Can I refuse to work?   
1  'Liz Truss the Brief?' World reacts to UK poli...   
2  Rationing energy is nothing new for off-grid c...   
3  The hunt for superyachts of sanctioned Russian...   
4  Platinum Jubilee: 70 years of the Queen in 70 ...   

                                           lowercase  
0                              can i refuse to work?  
1  'liz truss the brief?' world reacts to uk poli...  
2  rationing energy is nothing new for off-grid c...  
3  the hunt for superyachts of sanctioned russian...  
4  platinum jubilee: 70 years of the queen in 70 ...

In [8]:
# stop word removal
en_stopwords = stopwords.words('english')
titles['no_stopwords'] = titles['lowercase'].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)]))
titles.head()

title  \
0                              Can I refuse to work?   
1  'Liz Truss the Brief?' World reacts to UK poli...   
2  Rationing energy is nothing new for off-grid c...   
3  The hunt for superyachts of sanctioned Russian...   
4  Platinum Jubilee: 70 years of the Queen in 70 ...   

                                           lowercase  \
0                              can i refuse to work?   
1  'liz truss the brief?' world reacts to uk poli...   
2  rationing energy is nothing new for off-grid c...   
3  the hunt for superyachts of sanctioned russian...   
4  platinum jubilee: 70 years of the queen in 70 ...   

                                        no_stopwords  
0                                       refuse work?  
1  'liz truss brief?' world reacts uk political t...  
2    rationing energy nothing new off-grid community  
3      hunt superyachts sanctioned russian oligarchs  
4        platinum jubilee: 70 years queen 70 seconds

In [9]:
# punctuation removal
titles['no_stopwords_no_punct'] = titles.apply(lambda x: re.sub(r"([^\w\s])", "", x['no_stopwords']), axis=1)
titles.head()

title  \
0                              Can I refuse to work?   
1  'Liz Truss the Brief?' World reacts to UK poli...   
2  Rationing energy is nothing new for off-grid c...   
3  The hunt for superyachts of sanctioned Russian...   
4  Platinum Jubilee: 70 years of the Queen in 70 ...   

                                           lowercase  \
0                              can i refuse to work?   
1  'liz truss the brief?' world reacts to uk poli...   
2  rationing energy is nothing new for off-grid c...   
3  the hunt for superyachts of sanctioned russian...   
4  platinum jubilee: 70 years of the queen in 70 ...   

                                        no_stopwords  \
0                                       refuse work?   
1  'liz truss brief?' world reacts uk political t...   
2    rationing energy nothing new off-grid community   
3      hunt superyachts sanctioned russian oligarchs   
4        platinum jubilee: 70 years queen 70 seconds   

                               no_stopwords_no_punct  
0                                        refuse work  
1  liz truss brief world reacts uk political turmoil  
2     rationing energy nothing new offgrid community  
3      hunt superyachts sanctioned russian oligarchs  
4         platinum jubilee 70 years queen 70 seconds

In [10]:
# tokenize
titles['tokens_raw'] = titles.apply(lambda x: word_tokenize(x['title']), axis=1)
titles['tokens_clean'] = titles.apply(lambda x: word_tokenize(x['no_stopwords_no_punct']), axis=1)
titles.head()

title  \
0                              Can I refuse to work?   
1  'Liz Truss the Brief?' World reacts to UK poli...   
2  Rationing energy is nothing new for off-grid c...   
3  The hunt for superyachts of sanctioned Russian...   
4  Platinum Jubilee: 70 years of the Queen in 70 ...   

                                           lowercase  \
0                              can i refuse to work?   
1  'liz truss the brief?' world reacts to uk poli...   
2  rationing energy is nothing new for off-grid c...   
3  the hunt for superyachts of sanctioned russian...   
4  platinum jubilee: 70 years of the queen in 70 ...   

                                        no_stopwords  \
0                                       refuse work?   
1  'liz truss brief?' world reacts uk political t...   
2    rationing energy nothing new off-grid community   
3      hunt superyachts sanctioned russian oligarchs   
4        platinum jubilee: 70 years queen 70 seconds   

                               no_stopwords_no_punct  \
0                                        refuse work   
1  liz truss brief world reacts uk political turmoil   
2     rationing energy nothing new offgrid community   
3      hunt superyachts sanctioned russian oligarchs   
4         platinum jubilee 70 years queen 70 seconds   

                                          tokens_raw  \
0                      [Can, I, refuse, to, work, ?]   
1  ['Liz, Truss, the, Brief, ?, ', World, reacts,...   
2  [Rationing, energy, is, nothing, new, for, off...   
3  [The, hunt, for, superyachts, of, sanctioned, ...   
4  [Platinum, Jubilee, :, 70, years, of, the, Que...   

                                        tokens_clean  
0                                     [refuse, work]  
1  [liz, truss, brief, world, reacts, uk, politic...  
2  [rationing, energy, nothing, new, offgrid, com...  
3  [hunt, superyachts, sanctioned, russian, oliga...  
4  [platinum, jubilee, 70, years, queen, 70, seco...

In [11]:
# lemmatizing
lemmatizer = WordNetLemmatizer()
titles['tokens_clean_lemmatized'] = titles['tokens_clean'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
titles.head()

title  \
0                              Can I refuse to work?   
1  'Liz Truss the Brief?' World reacts to UK poli...   
2  Rationing energy is nothing new for off-grid c...   
3  The hunt for superyachts of sanctioned Russian...   
4  Platinum Jubilee: 70 years of the Queen in 70 ...   

                                           lowercase  \
0                              can i refuse to work?   
1  'liz truss the brief?' world reacts to uk poli...   
2  rationing energy is nothing new for off-grid c...   
3  the hunt for superyachts of sanctioned russian...   
4  platinum jubilee: 70 years of the queen in 70 ...   

                                        no_stopwords  \
0                                       refuse work?   
1  'liz truss brief?' world reacts uk political t...   
2    rationing energy nothing new off-grid community   
3      hunt superyachts sanctioned russian oligarchs   
4        platinum jubilee: 70 years queen 70 seconds   

                               no_stopwords_no_punct  \
0                                        refuse work   
1  liz truss brief world reacts uk political turmoil   
2     rationing energy nothing new offgrid community   
3      hunt superyachts sanctioned russian oligarchs   
4         platinum jubilee 70 years queen 70 seconds   

                                          tokens_raw  \
0                      [Can, I, refuse, to, work, ?]   
1  ['Liz, Truss, the, Brief, ?, ', World, reacts,...   
2  [Rationing, energy, is, nothing, new, for, off...   
3  [The, hunt, for, superyachts, of, sanctioned, ...   
4  [Platinum, Jubilee, :, 70, years, of, the, Que...   

                                        tokens_clean  \
0                                     [refuse, work]   
1  [liz, truss, brief, world, reacts, uk, politic...   
2  [rationing, energy, nothing, new, offgrid, com...   
3  [hunt, superyachts, sanctioned, russian, oliga...   
4  [platinum, jubilee, 70, years, queen, 70, seco...   

                             tokens_clean_lemmatized  
0                                     [refuse, work]  
1  [liz, truss, brief, world, reacts, uk, politic...  
2  [rationing, energy, nothing, new, offgrid, com...  
3  [hunt, superyachts, sanctioned, russian, oliga...  
4   [platinum, jubilee, 70, year, queen, 70, second]

In [12]:
# create lists for just our tokens
tokens_raw_list = sum(titles['tokens_raw'], []) #unpack our lists into a single list
tokens_clean_list = sum(titles['tokens_clean_lemmatized'], [])

### POS Tagging

In [13]:
nlp = spacy.load('en_core_web_sm')

#### Raw List

In [14]:
spacy_doc = nlp(' '.join(tokens_raw_list))

In [15]:
pos_df = pd.DataFrame(columns = ['token', 'pos_tag'])

In [16]:
for token in spacy_doc:
    pos_df = pd.concat([pos_df, pd.DataFrame.from_records(
        [{'token': token.text, 'pos_tag': token.pos_}]
    )], ignore_index=True)

In [18]:
pos_df_counts = pos_df.groupby(['token', 'pos_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
pos_df_counts.head(10)

token pos_tag  counts
95       :   PUNCT     543
8        '   PUNCT     300
2897    in     ADP     187
4082    to    PART     175
3268    of     ADP     172
22       -   PUNCT     166
4043   the     DET     163
1856   and   CCONJ     147
15      's    PART     143
97       ?   PUNCT     130

In [20]:
nouns = pos_df_counts[pos_df_counts.pos_tag == 'NOUN'][0:10]
nouns

token pos_tag  counts
4267     war    NOUN      35
3552  record    NOUN      15
3416  police    NOUN      14
4316     win    NOUN      14
4356    year    NOUN      14
3061  living    NOUN      13
4009     tax    NOUN      13
3368  people    NOUN      12
2326     day    NOUN      12
2031    boss    NOUN      11

In [21]:
verbs = pos_df_counts[pos_df_counts.pos_tag == 'VERB'][0:10]
verbs

token pos_tag  counts
3687    says    VERB      30
9          '    VERB      14
2670   found    VERB      13
4317     win    VERB      12
4324    wins    VERB      10
2713     get    VERB       9
2388    dies    VERB       9
3990    take    VERB       8
2982  killed    VERB       8
3745     set    VERB       8

In [22]:
adj = pos_df_counts[pos_df_counts.pos_tag == 'ADJ'][0:10]
adj

token pos_tag  counts
3244      new     ADJ      28
1400  Russian     ADJ      21
2606    final     ADJ      16
19          -     ADJ      14
2625    first     ADJ      12
3199     more     ADJ      10
1994      big     ADJ       9
2835     high     ADJ       9
3304    other     ADJ       8
3000     last     ADJ       8

#### Clean List

In [23]:
spacy_doc = nlp(' '.join(tokens_clean_list))
pos_df = pd.DataFrame(columns = ['token', 'pos_tag'])
for token in spacy_doc:
    pos_df = pd.concat([pos_df, pd.DataFrame.from_records(
        [{'token': token.text, 'pos_tag': token.pos_}]
    )], ignore_index=True)

In [24]:
pos_df_counts = pos_df.groupby(['token', 'pos_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
pos_df_counts.head(10)

token pos_tag  counts
30       2022     NUM      47
1162  england   PROPN      45
870       cup   PROPN      39
3707       uk   PROPN      37
3056      say    VERB      37
3840      war    NOUN      34
2386      new     ADJ      31
3948    world    NOUN      30
3949    world   PROPN      26
3710  ukraine   PROPN      23

In [25]:
nouns = pos_df_counts[pos_df_counts.pos_tag == 'NOUN'][0:10]
nouns

token pos_tag  counts
3840       war    NOUN      34
3948     world    NOUN      30
2136       man    NOUN      22
907        day    NOUN      21
3973      year    NOUN      20
1158    energy    NOUN      17
2847    record    NOUN      17
3870      week    NOUN      16
811       cost    NOUN      16
1130  election    NOUN      16

In [26]:
verbs = pos_df_counts[pos_df_counts.pos_tag == 'VERB'][0:10]
verbs

token pos_tag  counts
3056      say    VERB      37
3711  ukraine    VERB      22
3461     take    VERB      13
2133     make    VERB      13
1380    found    VERB      13
1459      get    VERB      13
358      beat    VERB      13
1651      hit    VERB      13
1473     give    VERB      11
3918      win    VERB      10

In [27]:
adj = pos_df_counts[pos_df_counts.pos_tag == 'ADJ'][0:10]
adj

token pos_tag  counts
2386      new     ADJ      31
3010  russian     ADJ      22
1324    first     ADJ      12
403       big     ADJ      12
1306    final     ADJ      11
2478     open     ADJ      11
1955     last     ADJ      10
3097   second     ADJ       8
2394     next     ADJ       8
3708  ukraine     ADJ       7

### NER

In [28]:
ner_df = pd.DataFrame(columns=['token', 'ner_tag'])

for token in spacy_doc.ents:
    if pd.isna(token.label_) is False:
        ner_df = pd.concat([ner_df, pd.DataFrame.from_records(
            [{'token': token.text, 'ner_tag': token.label_}]
        )], ignore_index=True)

In [29]:
ner_df.head()

token   ner_tag
0    russian      NORP
1    70 year      DATE
2  70 second      TIME
3       bull       ORG
4          1  CARDINAL

In [30]:
ner_df_counts = ner_df.groupby(['token', 'ner_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [31]:
ner_df_counts.head(10)

token   ner_tag  counts
34      2022  CARDINAL      30
451  russian      NORP      25
223    first   ORDINAL      15
35      2022      DATE      11
233   france       GPE      10
450   russia       GPE      10
519     tory      NORP       9
339   london       GPE       9
531       uk       GPE       8
4         10  CARDINAL       8

In [32]:
people = ner_df_counts[ner_df_counts.ner_tag == "PERSON"][:10]
people

token ner_tag  counts
432          putin  PERSON       7
127  boris johnson  PERSON       5
102    andy murray  PERSON       3
106  antonio conte  PERSON       3
120           beth  PERSON       2
292  john caldwell  PERSON       2
331    lenny henry  PERSON       2
252   harry meghan  PERSON       2
254     hodgkinson  PERSON       2
249          harry  PERSON       2